In [2]:
import cv2
import mediapipe as mp
import pyautogui

# Initialize webcam and mediapipe
webcam = cv2.VideoCapture(0)
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

# Create adjustable OpenCV window
cv2.namedWindow("Hand volume control using python", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Hand volume control using python", 800, 600)

while True:
    _, image = webcam.read()
    image = cv2.flip(image, 1)
    frame_height, frame_width, _ = image.shape
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    output = my_hands.process(rgb_image)
    hands = output.multi_hand_landmarks

    x1 = y1 = x2 = y2 = None

    if hands:
        for hand in hands:
            drawing_utils.draw_landmarks(image, hand)
            landmarks = hand.landmark
            for id, landmark in enumerate(landmarks):
                x = int(landmark.x * frame_width)
                y = int(landmark.y * frame_height)
                if id == 8:  # Index fingertip
                    cv2.circle(image, (x, y), 8, (0, 255, 255), -1)
                    x1, y1 = x, y
                if id == 4:  # Thumb tip
                    cv2.circle(image, (x, y), 8, (0, 255, 255), -1)
                    x2, y2 = x, y

        # Only proceed if both tips are detected
        if x1 is not None and x2 is not None:
            dist = (((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5) // 4
            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)
            print('Distance:', dist)  # Debug

            if dist > 20:
                pyautogui.press("volumeup")
            else:
                pyautogui.press("volumedown")

    cv2.imshow("Hand volume control using python", image)
    key = cv2.waitKey(10)
    if key == 27:  # ESC key
        break

webcam.release()
cv2.destroyAllWindows()
